In [36]:
import pandas as pd
import boto3
import json
import configparser

## Loading parameters from the config

In [41]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

#AWS Credentials
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

#IAM ROLE 
DWH_IAM_ROLE_NAME = config.get('CLUSTER', 'DWH_IAM_ROLE_NAME')

#CLUSTER
DWH_CLUSTER_IDENTIFIER = config.get('CLUSTER', 'DWH_CLUSTER_IDENTIFIER')
DWH_CLUSTER_TYPE = config.get('CLUSTER', 'DWH_CLUSTER_TYPE')
DWH_NODE_TYPE = config.get('CLUSTER', 'DWH_NODE_TYPE')
DWH_NUM_NODES = config.get('CLUSTER', 'DWH_NUM_NODES')

#DATABASE
DWH_DB = config.get('CLUSTER', 'DB_NAME')
DWH_DB_USER = config.get('CLUSTER', 'DB_USER')
DWH_DB_PASSWORD = config.get('CLUSTER', 'DB_PASSWORD')
DWH_PORT = config.get('CLUSTER', 'DB_PORT')

##  Importing the clients and resources

In [38]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Creating a role for the cluster and attaching a s3 read only policy to it

In [39]:
try:
    print("Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("Attaching Policy")

httpstatus = iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']
print(httpstatus)

print("Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
print(roleArn)

Creating a new IAM Role
Attaching Policy
200
Get the IAM role ARN
arn:aws:iam::559253790134:role/readfroms3


## creating the redshift cluster

In [42]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

### checking the status of the cluster

In [43]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
print(myClusterProps['ClusterStatus'])

creating


In [44]:
# needs while true loop
while True:
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    if myClusterProps['ClusterStatus'] == "available":
        break
print(myClusterProps['ClusterStatus'])

available


### getting the cluster endpoint and role arn

In [45]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  sparkify.cqa4vd5eex7x.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::559253790134:role/readfroms3


## For cleaning the resources

In [55]:
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

{'Cluster': {'ClusterIdentifier': 'sparkify',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'moataz',
  'DBName': 'sparkifydb',
  'Endpoint': {'Address': 'sparkify.cqa4vd5eex7x.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2022, 9, 17, 21, 10, 49, 107000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0400aa0192f780313',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-081a4aa5844425d7c',
  'AvailabilityZone': 'us-west-2d',
  'PreferredMaintenanceWindow': 'fri:08:30-fri:09:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 2,
 

In [58]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
print(myClusterProps['ClusterStatus'])

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster sparkify not found.

In [57]:
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)

{'ResponseMetadata': {'RequestId': '33ca016f-f30f-4546-8c4c-26073b0388a9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '33ca016f-f30f-4546-8c4c-26073b0388a9',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Sat, 17 Sep 2022 22:09:30 GMT'},
  'RetryAttempts': 0}}

# verifyng the tables

In [54]:
import psycopg2
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
#Credentials
host = config.get('CLUSTER','HOST')
dbname = config.get('CLUSTER','DB_NAME')
user = config.get('CLUSTER','DB_USER')
password = config.get('CLUSTER','DB_PASSWORD')
port = config.get('CLUSTER','DB_PORT')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(host, dbname, user, password, port))
cur = conn.cursor()


query1 = "select count(*) from staging_events where userId is not null"
query2 = "select * from staging_songs limit 1"
errorquery = "select colname, type, err_code, err_reason from stl_load_errors"

songplayq1 = "select * from songplays limit 1"
songplayq2 = "select count(*) from songplays"

userq1 = "select * from users limit 1"
userq2 = "select count(*) from users"

songq1 = "select * from songs limit 1"
songq2 = "select count(*) from songs"

artistq1 = "select * from artists limit 1"
artistq2 = "select count(*) from artists"

timeq1 = "select * from time limit 1"
timeq2 = "select count(*) from time"



def exco(q):
    cur.execute(q)
    conn.commit()
    results = cur.fetchone()
    while results:
        print(results)
        results = cur.fetchone()
    print("///////////////////////////////////////////////////////////")

exco(songplayq1)
exco(songplayq2)
print("########################################################################")

exco(userq1)
exco(userq2)
print("########################################################################")

exco(songq1)
exco(songq2)
print("########################################################################")

exco(timeq1)
exco(timeq2)


# exco(query1)
# exco(errorquery)





(26, datetime.datetime(2018, 11, 3, 17, 2, 24), '95', 'paid', None, None, 152, 'Winston-Salem, NC', '"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"')
///////////////////////////////////////////////////////////
(6820,)
///////////////////////////////////////////////////////////
########################################################################
('15', 'Lily', 'Koch', 'F', 'paid')
///////////////////////////////////////////////////////////
(104,)
///////////////////////////////////////////////////////////
########################################################################
('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546)
///////////////////////////////////////////////////////////
(24,)
///////////////////////////////////////////////////////////
########################################################################
(datetime.datetime(2018, 11, 3, 15, 54, 48), 1